In [ ]:
import time
import pandas as pd
import numpy as np
from multiprocessing.dummy import Pool
import time
import numpy as np
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import random
from bs4 import BeautifulSoup

In [ ]:
def get_stock_data(url, index_exchange):
    """
    Function to fetch stock information from a specified URL of the traderfox.de website
    with an specified index / exchange

    Inputs:
        url (str): one of the available urls from traderfox.de
        index_exchange (str): name of the index / exchnage from which
            the stocks should be selected
    """
    browser =  webdriver.Firefox()
    time.sleep(2)
    browser.get(url)
    time.sleep(5)
    soup = BeautifulSoup(browser.page_source,"html")
    time.sleep(5)
    names = []
    Row = soup.find('table', attrs = {'id' : 'insert-stocks'})
    for name in Row.find_all('td', attrs = {'class' : 'name'}):
            try:
                names.append(name.text)
            except:
                names.append("")

    wkns = []
    Row = soup.find('table', attrs = {'id' : 'insert-stocks'})
    for wkn in Row.find_all('td', attrs = {'data-id' : 'wkn'}):
            try:
                wkns.append(wkn.text)
            except:
                wkns.append("")

    FRAME = pd.DataFrame.from_dict(
            {'name': names,
            'wkn': wkns,
            'index' : index_exchange
            })
    time.sleep(1)
    browser.quit()
    return(FRAME)

In [ ]:

url_ls = ["https://traderfox.de/aktien/alle-amex-aktien-bestandteile"]#"https://traderfox.de/aktien/alle-nasdaq-aktien-bestandteile"]#]
#  "https://traderfox.de/aktien/alle-nyse-aktien-bestandteile"]#,
   # "https://traderfox.de/aktien/deutschland-160-bestandteile",
   # "https://traderfox.de/aktien/alle-nyse-aktien-bestandteile",
  #  "https://traderfox.de/aktien/alle-nasdaq-aktien-bestandteile",
#]
#https://traderfox.de/aktien/stoxx-europe-600-bestandteile
country_names_ls = ["AMEX"]

In [ ]:
ticker_frame = pd.DataFrame({})
for i, url in enumerate(url_ls):
    df = get_stock_data(url, country_names_ls[i])
    ticker_frame = ticker_frame.append(df)
    time.sleep(1)



In [ ]:
def get_ticker(initial_frame):
    url_ticker = "https://www.finanznachrichten.de/"
    url_yf_ticker = "https://finance.yahoo.com/quote/FB?p=FB"
    browser =  webdriver.Firefox()
    extension_path = r"C:\Users\Jonathan\AppData\Roaming\Mozilla\Firefox\Profiles\wl4weym2.default-1633941918487\extensions\jid1-MnnxcxisBPnSXQ@jetpack.xpi"
    browser.install_addon(extension_path, temporary=True)
    time.sleep(2)
    browser.get("about:support")
    time.sleep(5)
    browser.get(url_ticker)
    time.sleep(20)
    #clear = WebDriverWait(browser, 5).until(EC.presence_of_element_located((By.XPATH, "/html/body/div/div[2]/div[3]/div[2]/button"))).click()
    time.sleep(1)
    ticker = []
    de_ticker = []
    industry = []
    ISIN = []
    for i, wkn in enumerate(initial_frame.wkn):
        print(len(de_ticker), len(industry), len(ISIN))
        # if wkn not valid
        if wkn == "-":
            de_ticker.append("not_found")
            industry.append("not_found")
            ISIN.append("not_found")
            continue
        try:
            clear = WebDriverWait(browser, 25).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#fnk-suche-eingabe"))).clear()
            time.sleep(3)
            search = WebDriverWait(browser, 25).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#fnk-suche-eingabe")))
            time.sleep(1)
            search.send_keys(str(wkn))
            time.sleep(3)
            browser.find_element_by_css_selector('#suchhilfeListe > tbody:nth-child(2) > tr:nth-child(3) > td:nth-child(2)').click()
            time.sleep(random.randint(3,5))
            de_ticker.append(str(WebDriverWait(browser, 15).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#produkt-ticker"))).text.replace("Ticker-Symbol: ", "").lstrip()))
            #time.sleep(2)
        except:
            de_ticker.append("not_found")
            industry.append("not_found")
            ISIN.append("not_found")
            continue
        try:
            industry.append(str(WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".a > a:nth-child(2)"))).text))
            time.sleep(1)
        except:
            industry.append("not_found")
        try:
            ISIN.append(str(WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#produkt-isin"))).text.replace("ISIN: ", "").lstrip()))
            time.sleep(1)
            #time.sleep(random.randint(2,4))
        except:
            ISIN.append("not_found")


        if i % 4 == 0:
            try:
                browser.refresh()
                time.sleep(4)
            except:
                pass

    #check if lengths are equal   
    print(len(industry), len(ISIN), len(de_ticker))
    try:
        browser.get(url_yf_ticker)
        time.sleep(5)
    except:
        browser.get(url_yf_ticker)
        time.sleep(5)
    try:
        browser.find_element_by_css_selector("#scroll-down-btn").click()
        time.sleep(3)
    except:
        pass
    try:
        browser.find_element_by_css_selector("button.btn:nth-child(5)").click()
        time.sleep(4)
    except:
        pass
    for _ISIN in ISIN:
        if _ISIN == "not_found":
            ticker.append("not_found")
            continue
        time.sleep(2)
        #check again for popup and scroll
        try:
            browser.find_element_by_css_selector("#scroll-down-btn").click()
            time.sleep(3)
        except:
            pass
        try:
            browser.find_element_by_css_selector("button.btn:nth-child(5)").click()
            time.sleep(4)
        except:
            pass

        clear = WebDriverWait(browser, 25).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#yfin-usr-qry"))).clear()
        #time.sleep(1)
        try:
            search = WebDriverWait(browser, 25).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#yfin-usr-qry")))
            #time.sleep(1)
            search.send_keys(str(_ISIN))
            time.sleep(2)
            ticker.append(str(WebDriverWait(browser, 25).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#result-quotes-0 > div.modules_quoteLeftCol__gkCSv.modules_Ell__77DLP.modules_IbBox__2pmLe > div.modules_quoteSymbol__hpPcM.modules_Ell__77DLP.modules_IbBox__2pmLe"))).text))
        except:
            ticker.append("not_found")
            
    print(len(ticker))      

    frame = pd.DataFrame.from_dict(
        {"name" : initial_frame.name,
        "wkn" : initial_frame.wkn,
        "index" : "AMEX",
        'de_ticker' : de_ticker,
        'ticker': ticker,
        'industry': industry,
        'ISIN': ISIN
        })
    time.sleep(1)
    browser.quit()
    return frame


In [ ]:
data_ticker_amex = get_ticker(ticker_frame)


In [ ]:
data_ticker_amex.to_csv("AMEX_Stocks.csv")


-nasdaq 100

-nyse

-de

-Eurostoxx600


In [ ]:
# frame1 = pd.read_csv(r"C:\Users\Jonathan\Documents\GitHub\finance_Versuchskaninchen\fama_french\scraper\NASDAQStocks_p1.csv")
# frame2 = pd.read_csv(r"C:\Users\Jonathan\Documents\GitHub\finance_Versuchskaninchen\fama_french\scraper\NASDAQ_Stocks1197-2000.csv")
# frame3 = pd.read_csv(r"C:\Users\Jonathan\Documents\GitHub\finance_Versuchskaninchen\fama_french\scraper\NASDAQ_Stocks2000-rest.csv")

# total_frame = pd.concat([frame1, frame2, frame3])
# total_frame.to_csv("NASDAQ_all_Stocks.csv", encoding= 'utf8')

In [ ]:
def validate_ticker(path):

    """
    Function to update the stock information for the different exchanges / indicies
    This function is usefull to:
        - check if info is up to date
        - after adding new stocks to the {exchange / index}Stocks.csv files
        - to check that scraping was not corruped by bad internet connection or other issues
        - to validate the scraping
    
    Inputs:
        path (str): path of the file for which to perform the validation

    """
    url_ticker = "https://www.finanznachrichten.de/"
    url_yf_ticker = "https://finance.yahoo.com/quote/FB?p=FB"
    # browser =  webdriver.Firefox(executable_path='/home/jonathan/Schreibtisch/geckodriver')
    # extension_path = r"/home/jonathan/.mozilla/firefox/5xapxbqn.default-release/extensions/jid1-MnnxcxisBPnSXQ@jetpack.xpi"
    # browser.install_addon(extension_path, temporary=True)


    browser =  webdriver.Firefox()
    extension_path = r"C:\Users\Jonathan\AppData\Roaming\Mozilla\Firefox\Profiles\wl4weym2.default-1633941918487\extensions\jid1-MnnxcxisBPnSXQ@jetpack.xpi"
    browser.install_addon(extension_path, temporary=True)
    time.sleep(2)
    #extension_path = r"C:\Users\Jonathan\AppData\Roaming\Mozilla\Firefox\Profiles\wl4weym2.default-1633941918487\extensions\firefox-webext@zenmate.com.xpi"
    #browser.install_addon(extension_path, temporary=True)
    browser.maximize_window()
    browser.get("about:support")
    time.sleep(5)
    browser.get(url_ticker)
    time.sleep(5)
    file = pd.read_csv(path, encoding='utf-8')
    try:
        file.drop('Unnamed: 0', axis=1, inplace=True)
    except:
        pass
    try:
        file.drop('Unnamed: 0.1', axis = 1, index = True)
    except:
        pass

    for i, row in file.iterrows():
        if (row.ISIN) == "not_found" and (row.wkn != "-"):
            print(file.loc[i, "name"])
            try:
                time.sleep(1)
                clear = WebDriverWait(browser, 25).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#fnk-suche-eingabe"))).clear()
                time.sleep(random.randint(3,4))
                search = WebDriverWait(browser, 25).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#fnk-suche-eingabe")))
                time.sleep(random.randint(2,3))
                search.send_keys(str(row.wkn))
                time.sleep(3)
                browser.find_element_by_css_selector('#suchhilfeListe > tbody:nth-child(2) > tr:nth-child(3) > td:nth-child(2)').click()
                time.sleep(random.randint(2,3))
                file.at[i, "de_ticker"] = (WebDriverWait(browser, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#produkt-ticker"))).text.replace("Ticker-Symbol: ", "").lstrip())
                #time.sleep(2)
            except:
                pass
            try:
                if file.at[i, "industry"] == "not_found":
                    file.at[i, "industry"] = (WebDriverWait(browser, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".a > a:nth-child(2)"))).text)
                    time.sleep(3)
            except:
                pass
            try:
                if file.at[i, "ISIN"] == "not_found":
                    file.at[i, "ISIN"] = (WebDriverWait(browser, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#produkt-isin"))).text.replace("ISIN: ", "").lstrip())
                    time.sleep(random.randint(2,3))
            except:
                pass

    time.sleep(3)
    browser.get(url_yf_ticker)
    time.sleep(15)
    try:
        browser.find_element_by_css_selector("#scroll-down-btn").click()
        time.sleep(3)
    except:
        pass
    try:
        browser.find_element_by_css_selector("button.btn:nth-child(5)").click()
        time.sleep(4)
    except:
        pass
    for i, row in file.iterrows():
        if row.ISIN == "not_found":
            continue
        if file.at[i, "ticker"] == "not_found":
            time.sleep(random.randint(1,2))
            try:
                browser.find_element_by_css_selector("#scroll-down-btn").click()
                time.sleep(3)
            except:
                pass
            try:
                browser.find_element_by_css_selector("button.btn:nth-child(5)").click()
                time.sleep(4)
            except:
                pass

            clear = WebDriverWait(browser, 25).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#yfin-usr-qry"))).clear()
            time.sleep(random.randint(3,5))
            try:
                search = WebDriverWait(browser, 25).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#yfin-usr-qry")))
                time.sleep(1)
                search.send_keys(str(row.ISIN))
                time.sleep(5)
                file.at[i, "ticker"] = (WebDriverWait(browser, 25).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#result-quotes-0 > div.modules_quoteLeftCol__gkCSv.modules_Ell__77DLP.modules_IbBox__2pmLe > div.modules_quoteSymbol__hpPcM.modules_Ell__77DLP.modules_IbBox__2pmLe"))).text)
            except:
                pass
    #exchange = file.index[0]
    file.to_csv("val_" + path) 

    browser.quit()


In [ ]:
validate_ticker("val_AMEX_Stocks.csv")